In [18]:
!pip3 install selenium

     |████████████████████████████████| 904 kB 983 kB/s eta 0:00:01


In [79]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import re
import time

In [44]:
DRIVER_PATH = '../../chromedriver'
driver = webdriver.Chrome(executable_path=DRIVER_PATH)
driver.get('https://google.com')

In [9]:
url = 'https://www.gofundme.com/s?q=stop+asian+hate&pg={pg}'
base_url = url.format(pg=1)

In [28]:
driver.get(base_url)
nav = driver.find_elements_by_class_name('p-search-results-page-button')
num_pages = len(nav)

In [29]:
num_pages

56

In [64]:
fundraisers = []
for pg_no in range(1,num_pages+1):
    print('SCRAPING PAGE', pg_no)
    page_url = url.format(pg=pg_no)
    driver.get(page_url)
    funds = driver.find_elements_by_class_name('m-search-result-card')
    fund_urls = []
    for fund in funds:
        link = fund.find_element_by_tag_name('a')
        fund_urls.append(link.get_attribute("href"))
    for fund_url in fund_urls:
        try:
            driver.get(fund_url)
            money = driver.find_element_by_class_name('m-progress-meter-heading').text
            money_elements = money.split(' ')

            cur = 0
            goal = 0
            if ('raised' in money_elements):
                cur = int(re.sub("[^0-9]", "", money_elements[0]))
            else:
                goal = int(re.sub("[^0-9]", "", money_elements[0]))
            if ('of' in money_elements):
                goal = int(re.sub("[^0-9]", "", money_elements[money_elements.index('of')+1]))
            if (goal == 0):
                goal = cur
        
            created_date = driver.find_element_by_class_name('a-created-date').text.split('Created')[1][1:]
    #         print(cur, '/', goal, '-', created_date)
            fundraisers.append({
                'cur': cur,
                'goal': goal,
                'created': created_date
            })
        except:
            print('ERROR with fundraiser:', fund_url)

SCRAPING PAGE 1
SCRAPING PAGE 2
SCRAPING PAGE 3
SCRAPING PAGE 4
SCRAPING PAGE 5
SCRAPING PAGE 6
SCRAPING PAGE 7
SCRAPING PAGE 8
SCRAPING PAGE 9
SCRAPING PAGE 10
SCRAPING PAGE 11
SCRAPING PAGE 12
SCRAPING PAGE 13
SCRAPING PAGE 14
SCRAPING PAGE 15
SCRAPING PAGE 16
SCRAPING PAGE 17
ERROR with fundraiser: https://www.gofundme.com/f/original-character-and-charity-fund?qid=a3d773dee1f0cbb6eef588a4938145ff
SCRAPING PAGE 18
SCRAPING PAGE 19
SCRAPING PAGE 20
SCRAPING PAGE 21
SCRAPING PAGE 22
SCRAPING PAGE 23
SCRAPING PAGE 24
SCRAPING PAGE 25
SCRAPING PAGE 26
SCRAPING PAGE 27
SCRAPING PAGE 28
SCRAPING PAGE 29
SCRAPING PAGE 30
SCRAPING PAGE 31
SCRAPING PAGE 32
SCRAPING PAGE 33
SCRAPING PAGE 34
SCRAPING PAGE 35
SCRAPING PAGE 36
SCRAPING PAGE 37
SCRAPING PAGE 38
SCRAPING PAGE 39
SCRAPING PAGE 40
SCRAPING PAGE 41
SCRAPING PAGE 42
SCRAPING PAGE 43
SCRAPING PAGE 44
ERROR with fundraiser: https://www.gofundme.com/f/help-in-the-midst-of-tragedy?qid=9eee01d447dbf3203958a47ddd247c47
ERROR with fundraiser:

In [60]:
fundraisers

[{'cur': 93152, 'goal': 400000, 'created': 'July 7, 2021'},
 {'cur': 81800, 'goal': 100000, 'created': 'May 7, 2021'},
 {'cur': 1960, 'goal': 5000, 'created': 'July 14, 2021'},
 {'cur': 1386, 'goal': 2400, 'created': 'May 13, 2021'},
 {'cur': 4615, 'goal': 25000, 'created': 'May 5, 2021'},
 {'cur': 18536, 'goal': 20000, 'created': 'June 7, 2021'},
 {'cur': 255, 'goal': 1000, 'created': 'July 15, 2021'},
 {'cur': 11230, 'goal': 50000, 'created': 'June 1, 2021'},
 {'cur': 150, 'goal': 3000, 'created': 'July 8, 2021'},
 {'cur': 220, 'goal': 200, 'created': 'May 20, 2021'},
 {'cur': 170, 'goal': 500000, 'created': 'May 31, 2021'},
 {'cur': 48, 'goal': 25000, 'created': 'May 14, 2021'},
 {'cur': 3777, 'goal': 4000, 'created': 'May 20, 2021'},
 {'cur': 85, 'goal': 300, 'created': 'July 14, 2021'},
 {'cur': 6739, 'goal': 5000, 'created': 'June 13, 2021'},
 {'cur': 225, 'goal': 50000, 'created': 'July 7, 2021'},
 {'cur': 50, 'goal': 50000, 'created': 'May 5, 2021'},
 {'cur': 1405, 'goal': 2200

In [67]:
import json
with open('../jsons/fundraisers.json', 'w') as fout:
    json.dump(fundraisers , fout, indent=4)

In [115]:
DRIVER_PATH = '../../chromedriver'
driver = webdriver.Chrome(executable_path=DRIVER_PATH)

In [75]:
urls = {
    'Justice': 'https://www.gofundme.com/c/act/justice-aapi-community',
    'Safety': 'https://www.gofundme.com/c/act/safety-aapi-community',
    'Uplift': 'https://www.gofundme.com/c/act/uplift-aapi-community',
    'Support': 'https://www.gofundme.com/c/act/support-aapi-neighborhoods',
    'Atlanta': 'https://www.gofundme.com/c/act/atlanta-area-spa-shootings-fundraisers'
}
sections = urls.keys()

In [116]:
fundraisers = []
for section in sections:
    print('SECTION:', section)
    driver.get(urls[section])
    while(True):
        time.sleep(2)
        more_btn = driver.find_elements_by_class_name('js-load-more-results')
        if (len(more_btn) > 0):
            try:
                more_btn[0].click()
            except:
                print('all funds loaded')
                break
        else:
            break
    funds = driver.find_elements_by_class_name('react-campaign-tile')
    print('# of funds:', len(funds))
    fund_urls = []
    for fund in funds:
        fund_url = fund.find_element_by_tag_name('a')
        money = fund.find_elements_by_class_name('show-for-medium')[-1].text
        money_elements = money.split(' ')
        cur = 0
        goal = 0
        try:
            cur = int(re.sub("[^0-9]", "", money_elements[0]))
            goal = int(re.sub("[^0-9]", "", money_elements[money_elements.index('of')+1]))
        
        except:
            print('error with:', fund_url)
        fund_urls.append({
            'url': fund_url.get_attribute("href"),
            'cur': cur,
            'goal': goal,
        })
    for fund_url in fund_urls:
        try:
            driver.get(fund_url['url'])

            created_date = driver.find_element_by_class_name('a-created-date').text.split('Created')[1][1:]
    
            donors = 0
            try:
                driver.get(driver.current_url + '/donations')
                header = driver.find_element_by_class_name('o-modal-donations-header-title')
                text = header.find_element_by_tag_name('h2').text
                donors = text[text.find("(")+1:text.find(")")]
            except:
                print('no donors:', fund_url['url'])
    
            fundraisers.append({
                'cur': fund_url['cur'],
                'goal': fund_url['goal'],
                'donors': donors,
                'created': created_date,
                'section': section,
                'url': fund_url['url']
            })
        except:
            print('ERROR with fundraiser:', fund_url['url'])

SECTION: Justice
all funds loaded
# of funds: 43
{'url': 'https://www.gofundme.com/f/2b8zh292uo', 'cur': 997494, 'goal': 50000}
{'url': 'https://www.gofundme.com/f/help-yong-zheng-family-to-get-thru-this', 'cur': 353800, 'goal': 50000}
{'url': 'https://www.gofundme.com/f/help-and-support-we-are-with-you-vilma-kari', 'cur': 270458, 'goal': 20000}
{'url': 'https://www.gofundme.com/f/stopasianhatehalloffame', 'cur': 96441, 'goal': 250000}
{'url': 'https://www.gofundme.com/f/stop-the-hate-crime-chak-keung-chan-badly-beaten', 'cur': 94842, 'goal': 100000}
{'url': 'https://www.gofundme.com/f/assist-with-marcs-medical-expenses', 'cur': 93232, 'goal': 95000}
{'url': 'https://www.gofundme.com/f/84yearold-killed-in-san-francisco', 'cur': 91147, 'goal': 10000}
{'url': 'https://www.gofundme.com/f/GuanShuai', 'cur': 78055, 'goal': 250000}
{'url': 'https://www.gofundme.com/f/help-noel-quintana', 'cur': 53100, 'goal': 100000}
{'url': 'https://www.gofundme.com/f/support-inglemoor-hs-teacher-noriko-nas

SECTION: Uplift
all funds loaded
# of funds: 121
{'url': 'https://www.gofundme.com/f/qwajf-save-spoon-by-h', 'cur': 82366, 'goal': 80000}
{'url': 'https://www.gofundme.com/f/emergency-stirfry-mealsonwheels-for-seniors', 'cur': 40217, 'goal': 60000}
{'url': 'https://www.gofundme.com/f/w4wwoks-for-washington', 'cur': 18508, 'goal': 18000}
{'url': 'https://www.gofundme.com/f/fighting-covid19-antiasian-racism-with-research', 'cur': 17747, 'goal': 168000}
{'url': 'https://www.gofundme.com/f/k5u62p-covid19-mutual-aid-network-api-la', 'cur': 17418, 'goal': 17500}
{'url': 'https://www.gofundme.com/f/funding-openmeal', 'cur': 17095, 'goal': 10000}
{'url': 'https://www.gofundme.com/f/support-the-aacc', 'cur': 12110, 'goal': 35000}
{'url': 'https://www.gofundme.com/f/Donate3AF', 'cur': 10781, 'goal': 40000}
{'url': 'https://www.gofundme.com/f/figma-antiasian-violence-relief-fundraiser-drive', 'cur': 9335, 'goal': 6000}
{'url': 'https://www.gofundme.com/f/our-league-of-dreams-jacl-documentary', 'c

{'url': 'https://www.gofundme.com/f/1book1world-michigan-asian-american-book-drive', 'cur': 2741, 'goal': 20000}
{'url': 'https://www.gofundme.com/f/2cckshv5lc', 'cur': 2500, 'goal': 30000}
{'url': 'https://www.gofundme.com/f/crazy-talented-asiansfriends-flushing-town-hall', 'cur': 2436, 'goal': 5000}
{'url': 'https://www.gofundme.com/f/aanhpi-graduation-sashes-for-de-anza-high-school', 'cur': 1900, 'goal': 1900}
{'url': 'https://www.gofundme.com/f/2br53wnelc', 'cur': 1685, 'goal': 5000}
{'url': 'https://www.gofundme.com/f/support-our-local-asian-communities', 'cur': 1350, 'goal': 1000}
{'url': 'https://www.gofundme.com/f/will-bcc-stop-aapi-hate-call-to-action', 'cur': 1015, 'goal': 1000}
{'url': 'https://www.gofundme.com/f/newton-stands-with-aapi', 'cur': 930, 'goal': 2000}
{'url': 'https://www.gofundme.com/f/get-chinatown-vaccinated-initiative', 'cur': 600, 'goal': 10000}
{'url': 'https://www.gofundme.com/f/the-well-for-women-amplifying-aapi-womens-voices', 'cur': 286, 'goal': 1000}


In [118]:
with open('../jsons/official_fundraisers_donors.json', 'w') as fout:
    json.dump(fundraisers , fout, indent=4)

In [117]:
fundraisers

[{'cur': 997494,
  'goal': 50000,
  'donors': '32.9K',
  'created': 'March 18, 2021',
  'section': 'Justice',
  'url': 'https://www.gofundme.com/f/2b8zh292uo'},
 {'cur': 353800,
  'goal': 50000,
  'donors': '5.1K',
  'created': 'February 28, 2021',
  'section': 'Justice',
  'url': 'https://www.gofundme.com/f/help-yong-zheng-family-to-get-thru-this'},
 {'cur': 270458,
  'goal': 20000,
  'donors': '6.5K',
  'created': 'March 31, 2021',
  'section': 'Justice',
  'url': 'https://www.gofundme.com/f/help-and-support-we-are-with-you-vilma-kari'},
 {'cur': 96441,
  'goal': 250000,
  'donors': '149',
  'created': 'March 4, 2021',
  'section': 'Justice',
  'url': 'https://www.gofundme.com/f/stopasianhatehalloffame'},
 {'cur': 94842,
  'goal': 100000,
  'donors': '1.7K',
  'created': 'March 11, 2021',
  'section': 'Justice',
  'url': 'https://www.gofundme.com/f/stop-the-hate-crime-chak-keung-chan-badly-beaten'},
 {'cur': 93232,
  'goal': 95000,
  'donors': '2.1K',
  'created': 'February 17, 2021'